# Тест установки Ambrosia

Этот ноутбук проверяет работоспособность библиотеки Ambrosia после установки.

## 1. Проверка импорта

In [ ]:
import sys
print(f"Python version: {sys.version}")

import ambrosia
from ambrosia.designer import Designer
from ambrosia.splitter import Splitter
from ambrosia.tester import Tester
print(f"Ambrosia version: {ambrosia.__version__}")
print("✅ Импорт успешен!")

## 2. Подготовка тестовых данных

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'user_id': range(1000),
    'metric': np.random.normal(100, 15, 1000),
    'feature': np.random.rand(1000)
})

print(f"Размер данных: {df.shape}")
df.head()

## 3. Designer — расчёт размера выборки

In [ ]:
designer = Designer(dataframe=df, effects=1.05, metrics='metric')
result = designer.run('size')
print("Результат Designer (размер выборки):")
print(result)
print("\n✅ Designer работает!")

## 4. Splitter (hash) — разбиение на группы

In [ ]:
splitter = Splitter(dataframe=df, id_column='user_id', groups_size=500)
result_hash = splitter.run(method='hash', groups_number=2)
print("Распределение по группам (hash):")
print(result_hash['group'].value_counts())
print("\n✅ Splitter (hash) работает!")

## 5. Splitter (metric) — проверка hnswlib fallback

Этот тест проверяет, что metric split работает на macOS ARM благодаря fallback на hnswlib.

In [ ]:
splitter_metric = Splitter(dataframe=df, id_column='user_id', groups_size=500)
result_metric = splitter_metric.run(method='metric', groups_number=2, fit_columns=['feature'])
print("Распределение по группам (metric):")
print(result_metric['group'].value_counts())
print("\n✅ Splitter (metric) работает! (hnswlib fallback успешен)")

## 6. Tester — статистический тест

In [ ]:
df_test = pd.DataFrame({
    'user_id': range(200),
    'group': ['A'] * 100 + ['B'] * 100,
    'metric': np.concatenate([
        np.random.normal(100, 15, 100),
        np.random.normal(105, 15, 100)
    ])
})

tester = Tester(dataframe=df_test, column_groups='group')
result_test = tester.run(metrics='metric')
print("Результат Tester:")
print(result_test)
print("\n✅ Tester работает!")

## 7. Итог

In [ ]:
print("="*50)
print("РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ")
print("="*50)
print(f"Python: {sys.version.split()[0]}")
print(f"Ambrosia: {ambrosia.__version__}")
print()
print("✅ Импорт: OK")
print("✅ Designer: OK")
print("✅ Splitter (hash): OK")
print("✅ Splitter (metric): OK")
print("✅ Tester: OK")
print()
print("🎉 Все проверки пройдены успешно!")